Import the libraries for ETL, EDA and ML

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

I define a function to set the drive url to download csv files

In [2]:
def url_set(url: str):
    return 'https://drive.google.com/uc?id=' + url.split('/')[-2]

I create a function to generate the index id in the same where I create the dataframe of the platforms information.

In [3]:
def generate_id(show_id, platform):
    return platform[0] + show_id

dic_url = {"amazon_prime": url_set("https://drive.google.com/file/d/1LJIYUiPnFbU0mOKQMW54QZPPq1bAd5Zc/view?usp=share_link"),
            "disney_plus": url_set("https://drive.google.com/file/d/1d8BTVBj3NmCxUMknTkVcuVRWwVhXthiP/view?usp=share_link"),
           "hulu": url_set("https://drive.google.com/file/d/1Sy7HMCQgVlT31CAD2ewXrN82jMlsK21s/view?usp=share_link"),
           "netflix": url_set("https://drive.google.com/file/d/1yQ44qjfACWsR66lb-D_tDCmmDxPnIXvm/view?usp=share_link")}

df_names = pd.DataFrame()

for key, value in dic_url.items():
    # Get CSV file from Google Drive without downloading it
    df = pd.read_csv(value)
    df["id"] = df["show_id"].apply(lambda x: generate_id(x, key))
    df_names = pd.concat([df_names, df])

df_names.reset_index(drop=True, inplace=True)

list_url = [url_set("https://drive.google.com/file/d/1ImYbz29myZKGDZCYY4r5yzXWcdPqp-DL/view?usp=share_link"),
        url_set("https://drive.google.com/file/d/1rlpHWaxvo5kX5hyyOP7i5zpC5V9RfsaV/view?usp=share_link"),
        url_set("https://drive.google.com/file/d/1CBZA4xkDhfa-CX8dr92rhjKR3HhgctzE/view?usp=share_link"),
        url_set("https://drive.google.com/file/d/1CsaTyLVB-AZ78yJp9XeSw2qvTSNj9aYx/view?usp=share_link"),
        url_set("https://drive.google.com/file/d/1QztUrbE6CEC57AgbNcR9XJf9WIvwUF3P/view?usp=share_link"),
        url_set("https://drive.google.com/file/d/1y0TKNdKhSumjjaDSUGQTPIlMZuFqy1Zn/view?usp=share_link"),
        url_set("https://drive.google.com/file/d/18WgvpsLVK_5uhCJm5HyZSytNreOIRiT1/view?usp=share_link"),
        url_set("https://drive.google.com/file/d/1dwqAfTL7BXbOvJn_A3bwQkL9_gbIFoTz/view?usp=share_link")]

#Merging multiple csv files into a single dataframe
df_ratings = pd.concat(map(pd.read_csv, list_url))

df_ratings.reset_index(drop=True, inplace=True)

I replace the null values of the rating field with the string "G".

In [4]:
df_names["rating"] = df_names["rating"].fillna("G")

Change the date format to YYYYY-mm-dd

In [5]:
df_names["date_added"] = df_names["date_added"].str.strip()
df_names["date_added"] = pd.to_datetime(df_names["date_added"], format="%B %d, %Y")

In [6]:
df_ratings["timestamp"] = pd.to_datetime(df_ratings["timestamp"], unit="s")

The text fields are converted to lowercase

In [7]:
cols = ["type", "title", "director", "cast", "country", "listed_in", "description", "rating"]
df_names[cols] = df_names[cols].apply(lambda x: x.str.lower())

Split duration into two separate columns

In [8]:
df_duration = df_names["duration"].str.split(" ", n=1, expand=True)

Assign the resulting columns to duration_int and duration_type respectively

In [9]:
df_names["duration_int"] = df_duration[0].apply(lambda x: x if pd.notnull(x) else None)
df_names["duration_type"] = df_duration[1].apply(lambda x: x if pd.notnull(x) else None)

In [10]:
df_names['rating'].unique()

array(['g', '13+', 'all', '18+', 'r', 'tv-y', 'tv-y7', 'nr', '16+',
       'tv-pg', '7+', 'tv-14', 'tv-nr', 'tv-g', 'pg-13', 'tv-ma', 'pg',
       'nc-17', 'unrated', '16', 'ages_16_', 'ages_18_', 'all_ages',
       'not_rate', 'tv-y7-fv', 'not rated', '2 seasons', '93 min',
       '4 seasons', '136 min', '91 min', '85 min', '98 min', '89 min',
       '94 min', '86 min', '3 seasons', '121 min', '88 min', '101 min',
       '1 season', '83 min', '100 min', '95 min', '92 min', '96 min',
       '109 min', '99 min', '75 min', '87 min', '67 min', '104 min',
       '107 min', '84 min', '103 min', '105 min', '119 min', '114 min',
       '82 min', '90 min', '130 min', '110 min', '80 min', '6 seasons',
       '97 min', '111 min', '81 min', '49 min', '45 min', '41 min',
       '73 min', '40 min', '36 min', '39 min', '34 min', '47 min',
       '65 min', '37 min', '78 min', '102 min', '129 min', '115 min',
       '112 min', '61 min', '106 min', '76 min', '77 min', '79 min',
       '157 min', '28 mi

Normalize rating column

In [11]:
df_names["rating"] = df_names["rating"].astype(str)
for rating in df_names['rating'].unique():
    if rating.endswith('min'):
        duration_int, duration_type = rating.split(" ")
        filter = df_names['rating'] == rating
        df_names.loc[filter, 'duration_int'] = int(duration_int)
        df_names.loc[filter, 'duration_type'] = duration_type
        df_names.loc[filter, 'rating'] = 'unrated'
    elif rating.endswith('season') or rating.endswith('seasons'):
        filter = df_names['rating'] == rating
        df_names.loc[filter, 'rating'] = 'unrated'

df_names['rating'] = df_names['rating'].replace('all', 'all_ages')
df_names['rating'] = df_names['rating'].replace('nr', 'unrated')
df_names['rating'] = df_names['rating'].replace('tv-pg', 'pg')
df_names['rating'] = df_names['rating'].replace('tv-nr', 'unrated')
df_names['rating'] = df_names['rating'].replace('tv-g', 'g')
df_names['rating'] = df_names['rating'].replace('tv-ma', '18+')
df_names['rating'] = df_names['rating'].replace('nc-17', '18+')
df_names['rating'] = df_names['rating'].replace('16', '16+')
df_names['rating'] = df_names['rating'].replace('ages_16_', '16+')
df_names['rating'] = df_names['rating'].replace('18', '18+')
df_names['rating'] = df_names['rating'].replace('ages_18_', '18+')
df_names['rating'] = df_names['rating'].replace('not_rate', 'unrated')
df_names['rating'] = df_names['rating'].replace('not rated', 'unrated')
df_names['rating'] = df_names['rating'].replace('ur', 'unrated')

I eliminate the column "show_id", it does not provide more information than "id".

In [12]:
df_names.drop(columns="show_id", inplace=True)

I eliminate the "duration" column that separates your information in two.

In [13]:
df_names.drop(columns="duration", inplace=True)

I eliminate the "description" column, which is difficult to process and convert into a form that is suitable for the Machine Learning model.
Text processing may require more advanced techniques that are not feasible for the project in question.


In [14]:
df_names.drop(columns="description", inplace=True)

I remove the column "date_added", since the column "release_year" describes the year, then the date when the entry was added to the database does not provide additional information to the model.

In [15]:
df_names.drop(columns="date_added", inplace=True)

In [16]:
print(df_names.shape)
df_names.drop_duplicates(subset=["type", "title"] ,inplace=True)
print(df_names.shape)

(22998, 11)
(22179, 11)


In [17]:
print(df_ratings.shape)
df_ratings.drop_duplicates(inplace=True)
print(df_ratings.shape)

(11024289, 4)
(11024165, 4)


I remove the column "timestamp", since the column "release_year" describes the year, then the date when the entry was added to the database does not provide additional information to the model.

In [18]:
df_ratings.drop(columns='timestamp', inplace=True)

Group by movieId and average the scores of different users.

In [19]:
df_ratings_grouped = round(df_ratings.groupby('movieId', as_index=False)['rating'].mean(),1)

I change the name of the column to perform the merge.

In [20]:
df_ratings_grouped = df_ratings_grouped.rename(columns={'rating': 'scored'})

Merge with the general platform dataframe from the movieId.

In [21]:
df_names = pd.merge(df_names, df_ratings_grouped[['movieId', 'scored']], left_on='id', right_on='movieId')

I discard the "movieId" column that I used to perform the merge.

In [22]:
df_names.drop(columns="movieId", inplace=True)

I use the .info() method to know which columns have null values.

In [23]:
df_names.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22179 entries, 0 to 22178
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   type           22179 non-null  object 
 1   title          22179 non-null  object 
 2   director       14315 non-null  object 
 3   cast           17109 non-null  object 
 4   country        10795 non-null  object 
 5   release_year   22179 non-null  int64  
 6   rating         22179 non-null  object 
 7   listed_in      22179 non-null  object 
 8   id             22179 non-null  object 
 9   duration_int   21940 non-null  object 
 10  duration_type  21940 non-null  object 
 11  scored         22179 non-null  float64
dtypes: float64(1), int64(1), object(10)
memory usage: 2.2+ MB


I modify the formats of some columns of the platforms dataframe.

In [24]:
df_names["id"] = df_names["id"].astype(str)
df_names["type"] = df_names["type"].fillna("").astype(str)
df_names["country"] = df_names["country"].fillna("").astype(str)
df_names["director"] = df_names["director"].fillna("").astype(str)
df_names["cast"] = df_names["cast"].fillna("").astype(str)
df_names["rating"] = df_names["rating"].fillna("").astype(str)
df_names["duration_type"] = df_names["duration_type"].fillna("").astype(str)
df_names["release_year"] = df_names["release_year"].fillna(0).astype(int)
df_names["duration_int"] = df_names["duration_int"].fillna(0).astype(int)
df_names["scored"] = df_names["scored"].astype(float)


Number of users.

In [25]:
print('The ratings dataset has', df_ratings['userId'].nunique(), 'unique users')

The ratings dataset has 115077 unique users


Number of movies.

In [26]:
print('The ratings dataset has', df_ratings['movieId'].nunique(), 'unique movies')

The ratings dataset has 22998 unique movies


Number of ratings.

In [27]:
print('The ratings dataset has', df_ratings['rating'].nunique(), 'unique ratings')

The ratings dataset has 10 unique ratings


List of unique ratings.


In [28]:
print('The unique ratings are', sorted(df_ratings['rating'].unique()))

The unique ratings are [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


In [29]:
df_names.listed_in.unique()

array(['comedy, drama', 'drama, international', 'action, drama, suspense',
       ..., 'comedies, independent movies, thrillers',
       'international movies, lgbtq movies, romantic movies',
       'cult movies, dramas, thrillers'], dtype=object)

I remove the columns I do not use in the functions and create a dataset for the ML.

In [30]:
df_names.drop(columns=["director"], inplace=True)
df_ml = df_names.copy()
df_ml[df_ml["type"] == "movie"]
df_ml.drop(columns=["cast", "country", "release_year", "rating", "duration_int", "rating", "duration_type", "scored", "type"], inplace=True)

In [35]:
df_ml.drop(columns="id", inplace=True)

In [38]:
df_ml.listed_in.unique()

array(['comedy, drama', 'drama, international', 'action, drama, suspense',
       ..., 'comedies, independent movies, thrillers',
       'international movies, lgbtq movies, romantic movies',
       'cult movies, dramas, thrillers'], dtype=object)

In [66]:
df_ml.listed_in.value_counts()

drama                                    4038
comedy                                   3533
kids                                     1605
suspense                                 1498
documentary                              1090
                                         ... 
animals & nature, docuseries, medical       1
action & adventure, anime features          1
buddy, coming of age, drama                 1
animation, fantasy, musical                 1
animation, young adult audience             1
Name: listed_in, Length: 774, dtype: int64

In [43]:
df_ml.loc[df_ml['listed_in'].str.contains('dramas'), 'listed_in'] = 'drama'

In [47]:
df_ml.loc[df_ml['listed_in'].str.contains('lifestyle & culture'), 'listed_in'] = 'sports'

In [51]:
df_ml.loc[df_ml['listed_in'].str.contains('suspense'), 'listed_in'] = 'suspense'

In [53]:
df_ml.loc[df_ml['listed_in'].str.contains('comedy'), 'listed_in'] = 'comedy'

In [55]:
df_ml.loc[df_ml['listed_in'].str.contains('health & wellness'), 'listed_in'] = 'sports'

In [57]:
df_ml.loc[df_ml['listed_in'].str.contains('kids'), 'listed_in'] = 'kids'

In [59]:
df_ml.loc[df_ml['listed_in'].str.contains('science fiction'), 'listed_in'] = 'science fiction'


In [61]:
df_ml.loc[df_ml['listed_in'].str.contains('horror'), 'listed_in'] = 'horror'

In [63]:
df_ml.loc[df_ml['listed_in'].str.contains('western'), 'listed_in'] = 'western'


In [65]:
df_ml.loc[df_ml['listed_in'].str.contains('documentary'), 'listed_in'] = 'documentary'


In [67]:
df_ml.listed_in.nunique()

774

Maching Learning Model from TfidVectorizer and cosine_similarity is based on the construction of a feature matrix from the text documents of a dataset.

In [68]:
vectorizer = TfidfVectorizer()
#Construct the required TF-IDF matrix by fitting and transforming the data
vectorizer_matrix = vectorizer.fit_transform(df_names['listed_in'])

#Compute the cosine similarity matrix
cosine_sim = cosine_similarity(vectorizer_matrix)

#Define a function to get movie recommendations based on the cosine similarity score
def get_recommendations(title, cosine_sim=cosine_sim, df_names=df_names, top=5):
    #Get the index of the movie that matches the title
    idx = df_names[df_names['title'] == title].index[0]

    #Get the cosine similarity scores of all movies with the given movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    #Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #Loop until we have 5 recommended movies (excluding itself)
    movie_indices = []
    count = 0
    while len(movie_indices) < top:
        idx = sim_scores[count][0]
        if df_names.iloc[idx]['title'] != title: #Exclude the same movie
            movie_indices.append(idx)
        count += 1
    
    #Return the top n most similar movies
    return df_names.iloc[movie_indices]['title'].values

# get movie recommendations based on user input
title = "tarzan"
title = title.lower()
recommendations = get_recommendations(title)

# print the recommendations
print(f"Recommended movies based on {title}:")
for i, movie in enumerate(recommendations):
    print(f"{i+1}. {movie}")

Recommended movies based on tarzan:
1. on happiness road
2. invincible
3. metal skin panic madox-1
4. series before 1c onboarding - 2
5. nuttiest nutcracker


In [69]:
df_names.head()

,type,title,cast,country,release_year,rating,listed_in,id,duration_int,duration_type,scored
0,movie,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2014,g,"comedy, drama",as1,113,min,3.5
1,movie,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2018,13+,"drama, international",as2,110,min,3.5
2,movie,secrets of deception,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2017,g,"action, drama, suspense",as3,74,min,3.5
3,movie,pink: staying true,"interviews with: pink, adele, beyoncé, britney...",united states,2014,g,documentary,as4,69,min,3.5
4,movie,monster maker,"harry dean stanton, kieran o'brien, george cos...",united kingdom,1989,g,"drama, fantasy",as5,45,min,3.5


In [70]:
df_ml.head()

,title,listed_in,rating
0,the grand seduction,comedy,drama
1,take care good night,"drama, international",drama
2,secrets of deception,suspense,drama
3,pink: staying true,documentary,NaN
4,monster maker,"drama, fantasy",drama


In [37]:
df_ml.to_csv("df_ml.csv", index=False)

In [34]:
df_names.to_csv("df_platform.csv", index=False)
df_ratings.to_csv("df_rating.csv", index=False)
